In [1]:
import os
os.chdir('/home/ubuntu/nndl-project/')
from torch.utils.data import DataLoader, Dataset
from src.models.voxnet import VoxNet
from src.dataset import ModelNetDataset
from functools import partial
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
config = {
    'batch_size' : 10,
    'lr' : 0.01,
    'gamma' : 0.5    
}

#Datasets

metadata_path = '/home/ubuntu/nndl-project/data/modelnet10/metadata.parquet'
num_workers = 1

training_set = ModelNetDataset(metadata_path,split='train')
validation_set = ModelNetDataset(metadata_path,split='test')

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = DataLoader(training_set, batch_size=config['batch_size'], shuffle=True, num_workers=num_workers)
validation_loader = DataLoader(validation_set, batch_size=config['batch_size'], shuffle=False, num_workers=num_workers)

model = VoxNet()

# device
device = "cpu"

# LOSS FUNCTION
# no parameters to tune here
loss_c_fn = torch.nn.CrossEntropyLoss()
loss_o_fn = torch.nn.CrossEntropyLoss()



# OPTIMIZER
# tune learning rate
optimizer = torch.optim.Adam(model.parameters(),lr=config['lr'])



In [31]:
with torch.no_grad():

    v_data = next(iter(validation_loader))

    # get prediction and target
    v_voxels,v_o_y,v_y = v_data
    v_voxels = v_voxels.float().to(device)

    # we move these to INTEGER label because the loss accepts it; 
    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< FIX THIS IN DATASET SRC!
    v_y, v_o_y = v_y.float().to(device).argmax(1), v_o_y.float().to(device).argmax(1)
    v_o_y_pred, v_y_pred = model(v_voxels)
    v_o_y_pred, v_y_pred = v_o_y_pred.to(device), v_y_pred.to(device)


    # compute loss
    loss_c = loss_c_fn(v_y_pred, v_y)
    loss_o = loss_o_fn(v_o_y_pred,v_o_y)
    vtotal_loss = (1-config['gamma'])*loss_c + config['gamma']*loss_o

    # extract prediction from NN
    true_orientation=v_o_y
    predicted_orientation=v_o_y_pred.argmax(1)
    correct_orientation_prediction = true_orientation == predicted_orientation

    true_label=v_y
    predicted_label=v_y_pred.argmax(1)
    correct_label_prediction = true_label == predicted_label


tensor([False,  True, False, False, False, False, False, False, False, False])
tensor(1)
